In [1]:
import pandas as pd
import requests 
import sys

http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json

In [62]:
def api_BM(countries, indicator, startTime, endTime):
    """
    Importar múltiples series de la API del BM (Banco Mundial).
    
    Parámetros
    ----------
    countries: dict
        Diccionario de los códigos y nombres de los países.
        - countries = {
            "BR": "Brasil",
            "CL": "Chile",
            "PE": "Perú"
        }
        
    indicator: str
        
        Código del indicador o serie. 
        Se obtiene de: "https://datos.bancomundial.org/indicator/"
        
        - indicator = 'BX.KLT.DINV.WD.GD.ZS'
        
    startTime: str
        Fecha de inicio de la serie. 
        -Anual: yyyy
        
    endTime: str
        Fecha de fin de la serie. 
        -Anual: yyyy
        
    Retorno: 
    --------
    df: pd.DataFrame
        Países con las series consultadas.    
    
    Documentación:
    -------------
    https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structure
    
    
    @author: Mauricio Alvarado
             Norbert Andrei Romero Escobedo
    """

    
    keys = list(countries.keys())
    df = pd.DataFrame()
    
    for key in keys:
        url = f'http://api.worldbank.org/v2/country/{key}/indicator/{indicator}?format=json'
         
        response = requests.get(url) 
                
        if response.status_code == 200:
            pass
        else:
            print("Porfavor, revisa los datos ingresados")
            break
    
        response = response.json()
        observations = response[1]
    
        values_list = []
        time_list = []
    
        for obs in observations: 
            values_list.append((obs['value']))
            time_list.append(obs["date"])
    
        dictio = {
            "time": time_list, 
            f"{key}": values_list
        }
        dictio = pd.DataFrame(dictio)
                   
        if df.empty is True:
             df = pd.concat([df, dictio])
        else:
            df = pd.merge(df, dictio, how = "outer")
    
    df.set_index('time', inplace = True)
    df.sort_index(ascending=True, inplace=True)
    df.rename(countries, axis=1, inplace=True)
            
    return df
      
def get_codes (search_text):
        
    """Extraer metadatos
    Parámetros
    ----------
    search_text: list
        Dos palabras clave de la consulta. 
        
        -search_text =['life','expectancy']
        
    Retorno: 
    ---------
    df: pd. DataFrame
       Series consultadas
                  
    @author: Mauricio Alvarado
             Norbert Andrei Romero Escobedo  
    """

    formato = ["%20".join(search_text).lower() for i in search_text][0]   
   
    url = f"http://api.worldbank.org/v2/sources/2/search/{formato}?format=json"
    response = requests.get(url) 
    response = response.json()['source'][0]['concept'][1]['variable']#[0]['id']
    
    list_id= []
    list_names= []
    
    
    for i in response: 
        list_id.append(i['id'])
        list_names.append(i['name'])
        
        
    df= pd.DataFrame({
        "id":list_id,
        "title":list_names
    })

    return df

In [64]:
search_text = ['life','expectancy']
get_codes(search_text=search_text)


,id,title
0,SE.SCH.LIFE.FE,"Expected years of schooling, female"
1,SE.SCH.LIFE.MA,"Expected years of schooling, male"
2,SH.DYN.0509,Probability of dying among children ages 5-9 y...
3,SH.DYN.0514,"Probability of dying at age 5-14 years (per 1,..."
4,SH.DYN.1014,Probability of dying among adolescents ages 10...
5,SH.DYN.1019,Probability of dying among adolescents ages 10...
6,SH.DYN.1519,Probability of dying among adolescents ages 15...
7,SH.DYN.2024,Probability of dying among youth ages 20-24 ye...
8,SH.DYN.MORT,"Mortality rate, under-5 (per 1,000 live births)"
9,SH.DYN.MORT.FE,"Mortality rate, under-5, female (per 1,000 liv..."


In [25]:
search_text = ['hola', 'queso']
formato = ["%20".join(search_text).lower() for i in search_text][0]   
formato

'hola%20queso'

In [31]:
url = "http://api.worldbank.org/v2/sources/2/search/solid%20fuel?format=json"
response = requests.get(url) 
response = response.json()['source'][0]['concept'][0]['variable'][0]['id']
#response = response.json()['source'][0]['concept'][0]['variable'][i]
response
# variable = response.get("variable")
# variable

{'id': 'EG.NSF.ACCS.RU.ZS',
 'name': 'Access to non-solid fuel, rural (% of rural population)',
 'metatype': [{'id': 'IndicatorName',
   'value': 'Access to non-solid fuel, rural (% of rural population)'},
  {'id': 'Longdefinition',
   'value': 'Access to non-solid fuel, rural is the percentage of rural population with access to non-solid fuel.'},
  {'id': 'Statisticalconceptandmethodology',
   'value': 'Data for access to Non-Solid Fuel are collected among different sources: only data from nationally representative household surveys (including national censuses) were used. Survey sources include Demographic and Health Surveys (DHS) and Living Standards Measurement Surveys (LSMS), Multi-Indicator Cluster Surveys (MICS), the World Health Survey (WHS), other nationally developed and implemented surveys, and various government agencies (for example, ministries of energy and utilities). To develop the historical evolution of Non-Solid Fuel Use rates, a multi-level non-parametrical mixed mo

In [13]:
variable

In [3]:

indicator = 'BX.KLT.DINV.WD.GD.ZS'
startTime = '2000'
endTime = '2021'

countries = {
    "BR": "Brasil",
    "CL": "Chile",
    "PE": "Perú"
    }

api_BM(countries = countries, indicator = indicator, startTime = startTime, endTime = endTime)

,Brasil,Chile,Perú
time,,,
1972,0.787030,-0.009280,0.656331
1973,1.412451,-0.028510,0.869808
1974,1.100146,0.014250,0.450065
1975,1.007712,0.655977,0.480561
1976,1.015219,-0.009669,0.450779
1977,1.039445,0.150399,0.370031
1978,1.001605,1.131962,0.199684
1979,1.092898,1.119076,0.445586
1980,0.804993,0.733554,0.148391


##PORQUE CHUCHA SALE ESTO PORQUE PORQUE PORQUE PORQUE PORQUE PORQUE !!

In [30]:
base = 'http://api.worldbank.org/v2/country/'
countries = 'bra;chl'
indicator = 'BX.KLT.DINV.WD.GD.ZS?MRV=25'
url = f'{base}{countries}/indicators/{indicator}&format=json'
response = requests.get(url) 
response.status_code

response = response.json()[1][0]
response

{'indicator': {'id': 'BX.KLT.DINV.WD.GD.ZS',
  'value': 'Foreign direct investment, net inflows (% of GDP)'},
 'country': {'id': 'BR', 'value': 'Brazil'},
 'countryiso3code': 'BRA',
 'date': '2021',
 'value': 2.88636690274615,
 'unit': '',
 'obs_status': '',
 'decimal': 1}

In [41]:
country = 'CHL'
indicator = 'SP.DYN.LE00.IN'

df = pd.DataFrame()
url = f'http://api.worldbank.org/v2/country/{country}/indicator/{indicator}?format=json'
response = requests.get(url)
        
if response.status_code == 200:
    pass
else:
    print("Porfavor, revisa los datos ingresados")
    sys.exit
    
response = response.json()
observations = response[1]
    
values_list = []
time_list = []
    
for obs in observations: 
    values_list.append((obs['value']))
    time_list.append(obs["date"])
    
dictio = {"time": time_list, f"country": values_list}
df= pd.DataFrame(dictio)

df = df.set_index('time')  
df


,country
time,
2021,NaN
2020,79.377
2019,80.326
2018,80.133
2017,80.350
2016,80.079
2015,79.746
2014,79.473
2013,79.339
